In [1]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [2]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"
graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-03-10 17:14:52,608:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-03-10 17:14:52,609:connector._init_connection:99] - create driver connection  
[DEBUG:2023-03-10 17:14:53,615:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


In [3]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(max_length=30, trim_whitespaces=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class Roles(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty()
    }

class Programming(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
    
class Game(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
        
class HasSkill(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class HasRole(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    
    

In [4]:
p = Person.objects.create(first_name="rrmerugu")
print(p.id)

40988728


In [5]:
create_data = True

if create_data is True:
    Person.objects.delete()
    Programming.objects.delete()
    Game.objects.delete()
    HasSkill.objects.delete()
    HasRole.objects.delete()

    p = Person.objects.create(first_name="rrmerugu")

    _, django = Programming.objects.get_or_create(name="Django")
    _, python = Programming.objects.get_or_create(name="Python")
    _, neo4j = Programming.objects.get_or_create(name="neo4j")
    _, janusgraph = Programming.objects.get_or_create(name="Janusgraph")

    HasSkill.objects.get_or_create(p.id, django.id)
    HasSkill.objects.get_or_create(p.id, python.id)
    HasSkill.objects.get_or_create(p.id, neo4j.id)
    HasSkill.objects.get_or_create(p.id, janusgraph.id)

    _, cod = Game.objects.get_or_create(name="Call of Duty")
    _, fifa = Game.objects.get_or_create(name="Fifa")

    HasSkill.objects.get_or_create(p.id, cod.id)
    HasSkill.objects.get_or_create(p.id, fifa.id)


    _, dev_role = Roles.objects.get_or_create(name="Developer")
    _, admin_role = Roles.objects.get_or_create(name="Admin")

    HasRole.objects.get_or_create(p.id, dev_role.id)
    HasRole.objects.get_or_create(p.id, admin_role.id)


    print("Created data")

Created data


In [6]:
from gremlin_python.process.graph_traversal import __

p = Person.objects.get_or_none(has__first_name="rrmerugu")
print(p)
graph.connector.vertex.search(has__id=p.id).traverse_through("has_skill").toList()

<Node:Person id="41005280" is_active=True created_at=2023-03-10 17:15:13.848000 first_name=rrmerugu >


[<RelationShip:has_skill id="oef4s-oevvk-29ed-m7s" 41005280:Person -> has_skill -> 28792:Programming >,
 <RelationShip:has_skill id="oeh3w-oevvk-29ed-18fc" 41005280:Person -> has_skill -> 57576:Game >,
 <RelationShip:has_skill id="oefx8-oevvk-29ed-270g" 41005280:Person -> has_skill -> 102400:Programming >,
 <RelationShip:has_skill id="oegpo-oevvk-29ed-2a68" 41005280:Person -> has_skill -> 106496:Game >,
 <RelationShip:has_skill id="oefj0-oevvk-29ed-oez1c" 41005280:Person -> has_skill -> 41009376:Programming >,
 <RelationShip:has_skill id="oegbg-oevvk-29ed-of274" 41005280:Person -> has_skill -> 41013472:Programming >]

In [8]:
# from gremlin_python.process.graph_traversal import __

# p = Person.objects.get_or_none(has__first_name="rrmerugu")
# print(p)
# graph.connector.vertex.search(has__id=p.id).traverse_through("has_skill").to("Programming").toList()